In [ ]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI


In [ ]:
load_dotenv(override=True)
api_key=os.getenv("OPENAI_API_KEY")

if not api_key:
 print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
 print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip()!=api_key:
 print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them")
else:
 print("API key found and looks good so far!") 

In [ ]:
# imports

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [ ]:
# Constants

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [ ]:
# Create a messages list using the same format that we used for OpenAI

messages = [
    {"role": "user", "content": "Describelangflow"}
]

In [ ]:
payload = {
        "model": MODEL,
        "messages": messages,
        "stream": False
    }

In [ ]:
response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
print(response.json()['message']['content'])

In [ ]:
import ollama

response = ollama.chat(model=MODEL, messages=messages)
print(response['message']['content'])

In [ ]:
# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [ ]:
ed = Website("https://edwarddonner.com")
print(ed.title)
print(ed.text)

In [ ]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = Website(url)
    response = ollama.chat(
        model = "llama3.2",
        messages = messages_for(website)
    )
    return response['message']['content']

In [ ]:
summarize("https://edwarddonner.com")

In [ ]:
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [ ]:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [ ]:
print(user_prompt_for(ed))

In [ ]:
messages = [
    {"role": "system", "content": "You are a snarky assistant"},
    {"role": "user", "content": "What is 2 + 2?"}
]

In [ ]:
 response = ollama.chat(
        model = "llama3.2",
        messages = messages
    )
 print(response['message']['content'])

In [ ]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [ ]:
messages_for(ed)

In [ ]:
def summarize(url):
    website = Website(url)
    response = ollama.chat(
        model = "llama3.2",
        messages = messages_for(website)
    )
    return response['message']['content']

In [ ]:
summarize("https://edwarddonner.com")

In [ ]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [ ]:
display_summary("https://edwarddonner.com")